In [1]:
#INSTALLATIONS AND IMPORTS

#pip install opencv-python

import numpy as np
import cv2
import sys

#IMPORT IMAGE
img = cv2.imread('1.png')

In [2]:
#define a function that converts the mm of coinage to pounds
def currencyvalue(r_mm):
    accuracy=0.5
    
    poundmm=1.2
    poundmaxmm=poundmm+accuracy
    poundminmm=poundmm-accuracy
    
    if r_mm > poundminmm and r_mm < poundmaxmm:
        return 1
    
    
    twentypmm=1.16
    twentypmaxmm=twentypmm+accuracy
    twentypminmm=twentypmm-accuracy
    
    if r_mm > twentypminmm and r_mm < twentypmaxmm:
        return 0.20
    
    
    onepmm=1.11
    onepmaxmm=onepmm+accuracy
    onepminmm=onepmm-accuracy
    
    if r_mm > onepminmm and r_mm < onepmaxmm:
        return 0.01
    
    
    twopmm=1.37
    twopmaxmm=twopmm+accuracy
    twopminmm=twopmm-accuracy
    
    if r_mm > twopminmm and r_mm < twopmaxmm:
        return 0.02
    
    
    fivepmm=0.99
    fivepmaxmm=fivepmm+accuracy
    fivepminmm=fivepmm-accuracy
    
    if r_mm > fivepminmm and r_mm < fivepmaxmm:
        return 0.05
    
    tenpmm=1.35
    tenpmaxmm=tenpmm+accuracy
    tenpminmm=tenpmm-accuracy
    
    if r_mm > tenpminmm and r_mm < tenpmaxmm:
        return 0.10
    
    
    fiftypmm=1.45
    fiftypmaxmm=fiftypmm+accuracy
    fiftypminmm=fiftypmm-accuracy
    
    if r_mm > fiftypminmm and r_mm < fiftypmaxmm:
        return 0.50
    
    
    twopoundmm=1.5
    twopoundmaxmm=twopoundmm+accuracy
    twopoundminmm=twopoundmm-accuracy
    
    if r_mm > twopoundminmm and r_mm < twopoundmaxmm:
        return 2

In [3]:
#IMAGE PREPROCESSES 

#copy of original image for comparison
output = img.copy()

#contrast control
alpha = 1.5 # Contrast control (1.0-3.0)
beta = 40 # Brightness control (0-100)

adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

#code to observe contrast changes (comment out when not necessary)
#cv2.imshow('original', img)
#cv2.imshow('adjusted', adjusted)
#cv2.waitKey()

#Convert image to Grayscale
adjusted = cv2.cvtColor(adjusted,cv2.COLOR_BGR2GRAY)

#Blur image
adjusted = cv2.GaussianBlur(adjusted, (31,31), 0)

#Threshold
#adaptive technique
adjusted = cv2.adaptiveThreshold(adjusted,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
                   cv2.THRESH_BINARY,51,2)

# Creating kernel
kernel = np.ones((5, 5), np.uint8)
  
# Using cv2.erode() method 
adjusted = cv2.erode(adjusted, kernel, cv2.BORDER_REFLECT) 

#binary technique
#ret, adjusted = cv2.threshold(adjusted,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#otsu technique
#ret,adjusted = cv2.threshold(adjusted,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#Canny edge detection
#adjusted = cv2.Canny(adjusted, 50, 200)

cv2.imshow('post process', adjusted)



In [ ]:
# # detect circles
circles = cv2.HoughCircles(image=adjusted,
                            method=cv2.HOUGH_GRADIENT, 
                            dp=0.5, 
                            param1=100,
                            param2=25, 
                            minDist=50, 
                            minRadius=1, 
                            maxRadius=100)

rcircles = np.uint16(np.around(circles))

if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    print ("Number of circles:", len(circles))

    coins = np.zeros(len(circles))
    
    # Count circles
    count=1 
    for (x, y, r) in circles:
        #Calculate radius in mm:
        r_mm = round(r/52, 2)
        # Create outer circle
        cv2.circle(output, (x,y), r, (0, 0, 0), 1)
        # Create center rectangle
        cv2.rectangle(output, (x-2, y-2), (x+2, y+2), (0,255,0), -1)
        # Add radius to center
        cv2.putText(output, str(r_mm), 
                    (x-15, y-5), 
                    cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                    0.7, (0, 0, 0), 1)
        # Print the radius of detected circles in pixels and mm
        print ('c' + str(count) + ' ' + str(r_mm) + ' ' + str(currencyvalue(r_mm)))
        coins[count-1] = currencyvalue(r_mm)
        count += 1
    
    print("total value=", np.sum(coins))
    cv2.imshow("output", np.hstack([img, output]))
    cv2.waitKey(0)
    
 
  

Number of circles: 9
c1 1.44 1
c2 1.54 1
c3 1.4 1
c4 1.21 1
c5 1.13 1
c6 1.38 1
c7 1.19 1
c8 1.02 1
c9 1.23 1
total value= 9.0
